In [ ]:
import tensorflow as tf 
from tensorflow import keras 
from keras import layers 

In [ ]:
class EncoderTransFormer(layers.Layer):
  def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
    super(EncoderTransFormer, self).__init__()

    self.att = layers.MultiHeadAttention(num_heads=num_heads,
                                         key_dim=embed_dim)
    self.ffn = keras.Sequential([
        layers.Dense(ff_dim , activation='relu'),
        layers.Dense(embed_dim),
    ])
    self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = layers.Dropout(rate)
    self.dropout2 = layers.Dropout(rate)

  
  def call(self, inputs, training):
    
    attn_output = self.att(inputs, inputs)
    attn_output = self.dropout1(attn_output, training=training)
    out1 = self.layernorm1(inputs + attn_output)
    ffn_output = self.ffn(out1)
    ffn_output = self.dropout2(ffn_output, training=training)
    out2 = self.layernorm2(out1 + ffn_output)
    print("",self.layernorm2(out1 + ffn_output))

    return out2 
  
  

In [ ]:
class ModelTransFormer(layers.Layer):
    def __init__(self, embed_dim , num_heads , ff_dim , rate=0.1):
        super().__init__()
        self.embed_dim = embed_dim
        self.num_head = num_heads 
        self.ff_dim = ff_dim 
        self.attn = layers.MultiHeadAttention(num_heads=num_heads , key_dim=embed_dim)
        self.attn1 = layers.MultiHeadAttention(num_heads=num_heads , key_dim=embed_dim)
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.ffn = keras.Sequential([
            layers.Dense(ff_dim, activation='relu'),
            layers.Dense(embed_dim),
        ])
        self.layernorm3 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)
        self.dropout3 = layers.Dropout(rate)
        
        # Thêm một lớp mật độ để chuyển đổi kích thước của tensor đầu ra của lớp mã hóa
        # Giả sử bạn muốn có 64 đơn vị ẩn cho lớp chú ý nhiều đầu
        self.dense = layers.Dense(64, activation='relu')

    # def call(self, inputs , training):
    #     encoder = EncoderTransFormer(embed_dim=self.embed_dim,
    #                                 num_heads=self.num_head, ff_dim=self.ff_dim )
    #     # đầu ra encoder shape tensor=(none , 200 ,32)
    #     encoder_output = encoder(inputs)
        
    #     # Biến đổi kích thước của tensor đầu ra của lớp mã hóa bằng cách sử dụng lớp mật độ
    #     encoder_output = self.dense(encoder_output)
        
    #     attn_output = self.attn(inputs, inputs)
    #     attn_output = self.dropout1(attn_output , training=training)
        
    #     # đầu ra của lớp layer1 decoder shape tensor= (none ,200 , 32)
    #     out1 = self.layernorm1(inputs + attn_output)

    #     # Sử dụng tensor đã được biến đổi làm Q, K và V cho lớp chú ý nhiều đầu thứ hai
    #     attn_output1 = self.attn1(encoder_output , encoder_output , out1)

    #     attn_output1 = self.dropout2(attn_output1 , training=training)
    #     out2 = self.layernorm2(out1 + attn_output1)
    #     ffn_output = self.ffn(out2)
    #     fn_output = self.dropout3(ffn_output, training=training)

    #     print("",out1.shape)
    #     return self.layernorm3(out2 + fn_output)
    def call(self, inputs , training, mask=None):
        if mask is not None :
            mask = mask[:, tf.newaxis, :] *1e-9

        attn_output = self.attn(inputs, inputs)
        attn_output = self.dropout1(attn_output , training=training)

        out1 = self.layernorm1(inputs + attn_output)

        # Sử dụng tensor đã được biến đổi làm Q, K và V cho lớp chú ý nhiều đầu thứ hai
        attn_output1 = self.attn1(inputs , inputs , out1 , attention_mask=mask)

        attn_output1 = self.dropout2(attn_output1 , training=training)
        
        
        # Cộng hai tensor lại và truyền chúng vào lớp layernorm2
        out2 = self.layernorm2(out1 + attn_output1)

        ffn_output = self.ffn(out2)
        fn_output = self.dropout3(ffn_output, training=training)
        
        

        print("",out1.shape)
        return self.layernorm3(out2 + fn_output)

In [ ]:
class TokenPositionEmbedding(layers.Layer):
    def __init__(self, maxlen , vocab_size , embed_dim):
        super().__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size ,output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen , output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0 , limit=maxlen , delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions 
     

In [ ]:
vocab_size = 20000  # Only consider the top 20k words
maxlen = 200  # Only consider the first 200 words of each movie review
(x_train, y_train), (x_val, y_val) = keras.datasets.imdb.load_data(num_words=vocab_size)
print(len(x_train), "Training sequences")
print(len(x_val), "Validation sequences")
x_train = keras.utils.pad_sequences(x_train, maxlen=maxlen)
x_val = keras.utils.pad_sequences(x_val, maxlen=maxlen)

In [ ]:
embed_dim = 32 # embeding size for each token 
num_heads = 2 # number of attention heads 
ff_dim = 32 # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenPositionEmbedding(maxlen, vocab_size, embed_dim)

x = embedding_layer(inputs)
# encoder=EncoderTransFormer(embed_dim , num_heads, ff_dim)
Transformer = ModelTransFormer(embed_dim , num_heads, ff_dim)
x = Transformer (x)
# x = encoder(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20 , activation='relu')(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(2 , activation='softmax')(x)


model = keras.Model(inputs=inputs , outputs=outputs)


In [ ]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(
    x_train, y_train, batch_size=32, epochs=3, validation_data=(x_val, y_val)
)